In [2]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

### Dummy Data

In [3]:
num_weeks = 24
news_dim = 5    # each weekly news vector is 5-dimensional
days_per_week = 7

# Create a 2D array for weekly news, shape (24, 5)
weekly_news = np.random.randn(num_weeks, news_dim)

# Expand the weekly news matrix to daily news:
daily_news = np.repeat(weekly_news, repeats=days_per_week, axis=0)  # shape: (24*7, 5)

print("Daily news array (24 * 7 weeks, 5 dimensions):")
#print(weekly_news)
print("Shape:", daily_news.shape)

# For daily stock prices:
days_per_week = 7
total_days = num_weeks * days_per_week  # 24 weeks * 7 days = 168 days

# Create a daily stock price vector; here, we assume each day's stock price is 1-dimensional.
daily_stock_prices = abs(np.random.randn(total_days, 1)) + 10
print("\nDaily stock price array (168 days, 1 dimension):")
#print(daily_stock_prices)
print("Shape:", daily_stock_prices.shape)

Daily news array (24 * 7 weeks, 5 dimensions):
Shape: (168, 5)

Daily stock price array (168 days, 1 dimension):
Shape: (168, 1)


We now have 168 data points as the sequence. Each point has one exogenous variable, news embedding, and one stock price.

In [4]:
#Making the sequence for sequence based predictors
seq = np.concatenate([daily_news, daily_stock_prices], axis=1)  # Resulting shape: (168, 6)

In [5]:
stock_1d = daily_stock_prices.reshape(-1)  # Flatten from (168,1) to (168,)
labels = np.zeros(total_days - 1, dtype=int)  # We'll have 167 valid comparisons

for t in range(total_days - 1):
    if stock_1d[t+1] > stock_1d[t]:
        labels[t] = 1
    else:
        labels[t] = 0

print("\nLabel vector shape:", labels.shape)  # (167,)


Label vector shape: (167,)


In [6]:
X = seq[:-1]       # shape (167, 6)
y = labels         # shape (167,)

In [26]:
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(0)  # shape: (1, 167, 6)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(0)     # shape: (1, 167)

In [27]:
X_tensor.shape

torch.Size([1, 167, 6])

### RNN (without Teacher Forcing)

Since we are doing classification at each time point, we are not using the output of the model as an input to the next timestep.

In [28]:
class StockRNN(nn.Module):
    def __init__(self,  input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x) #Dont need the hidden state here
        logits = self.fc(out)

        return logits

In [29]:
# Model hyperparameters
input_dim = 6      # Daily feature vector dimension.
hidden_dim = 32    # Hidden state size.

In [30]:
model = StockRNN(input_dim, hidden_dim)

In [31]:
criterion = nn.BCEWithLogitsLoss()  # Combines sigmoid + binary cross-entropy.
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
num_epochs = 100

for epoch in range(num_epochs):
    
    model.train()
    optimizer.zero_grad()
    
    # Forward pass: model returns logits of shape (1, 167, 1)
    logits = model(X_tensor)  # shape: (1, 167, 1)
    # Flatten logits and targets to (batch*seq_len, 1)
    logits = logits.view(-1)  # shape: (167,)
    targets = y_tensor.view(-1)  # shape: (167,)
    
    loss = criterion(logits, targets)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item():.4f}")

Epoch 10/100 - Loss: 0.6907
Epoch 20/100 - Loss: 0.6900
Epoch 30/100 - Loss: 0.6870
Epoch 40/100 - Loss: 0.6854
Epoch 50/100 - Loss: 0.6836
Epoch 60/100 - Loss: 0.6820
Epoch 70/100 - Loss: 0.6804
Epoch 80/100 - Loss: 0.6789
Epoch 90/100 - Loss: 0.6775
Epoch 100/100 - Loss: 0.6761


In [33]:
last_feature = seq[-1]  # shape: (6,)

# Convert it to a tensor and reshape so that it matches the model's expected input shape.
# Expected shape: (batch_size, seq_length, input_dim)
# Here, batch_size = 1 and seq_length = 1.
last_feature_tensor = torch.tensor(last_feature, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

In [34]:
model.eval()
with torch.no_grad():
    # Pass the last element through the model.
    # For our RNN (StockRNN_Sigmoid), the output will have shape (1, 1, 1)
    logit = model(last_feature_tensor)
    # Apply sigmoid to convert the logit into a probability.
    prediction_prob = torch.sigmoid(logit)
    # Convert to a Python scalar.
    prediction_prob = prediction_prob.item()
    
print("Prediction probability for the next day:", prediction_prob)

Prediction probability for the next day: 0.5639985203742981
